In [7]:
import torch
from torch import optim,nn
from torch.autograd import Variable
import numpy as np
import time
import os
from models.data_loader import DataLoader
from models.retain_bidirectional import RETAIN

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [4]:
# hyperparameters
batch_size = 50
max_seq_length = 300
min_seq_length = 5
num_classes = 267+1
cuda_flag = True

In [5]:
# data loader
D = DataLoader(batch_size=batch_size,
   data_dir='data/batches/',
    mode='val', max_seq_length=max_seq_length, min_seq_length=min_seq_length)


RETAIN (
  (RNN1): RNN(128, 128, batch_first=True, bidirectional=True)
  (RNN2): RNN(128, 128, batch_first=True, bidirectional=True)
  (wa): Linear (256 -> 1)
  (Wb): Linear (256 -> 128)
  (W_out): Linear (128 -> 268)
)

In [14]:
import numpy as np
for num in np.arange(500,9500,500):
    ver = 'retain_bi_%d.pth'%num
    model = torch.load(os.path.join('/home/mjc/github/EHRVis/data/saved_weights',ver))
    model.eval()
    # test model
    cnt = 0
    total = 0
    correct = 0
    correct_5 = 0
    for i,file in enumerate(D.val_list[5:10]):
    # for i,file in enumerate(['2014_42.pckl']):
    #     print('[%d/%d] opening from %s'%(i+1,len(D.test_list),file))
        file_num = int(file.split('_')[1].split('.')[0])
        D.batch_size = int(20000/file_num)
        D.load_batch_file(file)
        for i in range(D.batch_count):
            cnt+=1
            input_list, targets = D.get_batch()
            inputs = model.list_to_tensor(input_list)
            outputs = model(inputs)
            targets = torch.LongTensor(targets).view(len(inputs),-1)[:,-1] # to only use last of each sequence
            if cuda_flag:
                targets = targets.cuda()
            total+=len(inputs)
            targets = targets.tolist()
            for tup in zip(targets,outputs.topk(5)[1].data.tolist()):
                if tup[0]==tup[1][0]:
                    correct +=1
                if tup[0] in tup[1]:
                    correct_5 +=1
    print(ver)
    print("Top-1 accuracy: %d/%d = %1.3f"%(correct,total,correct*1.0/total))
    print("Top-5 accuracy: %d/%d = %1.3f"%(correct_5,total,correct_5*1.0/total))

FileNotFoundError: [Errno 2] No such file or directory: '/home/mjc/github/EHRVis/data/saved_weights/retain_bi_500.pth'